In [1]:
from msgs import MAVLink_heartbeat_message

In [3]:
# __init__(self, type: int, autopilot: int, base_mode: int, custom_mode: int, system_status: int, mavlink_version: int)
MAV_TYPE = 2
MAV_AUTOPILOT = 1
MAV_MODE = 0
MAV_STATE = 0
MAVLINK_VERSION = 1
CRC_EXTRA_HEARTBEAT = 50
CRC_EXTRA_COMMAND_LONG = 20
CRC_EXTRA_COMMAND_ACK = 143

msg = MAVLink_heartbeat_message(type=MAV_TYPE, autopilot=MAV_AUTOPILOT, base_mode=MAV_MODE, custom_mode=MAV_MODE, system_status=MAV_STATE, mavlink_version=MAVLINK_VERSION)
for i in range(1):
    print(msg.pack())

b'\xfe\t\x00\x01\x01\x00\x00\x00\x00\x00\x02\x01\x00\x00\x01\x89R'


# Testing decoder

In [4]:
from decoder import decoder

dec = decoder()
# known msgs ------------------------>
# from_msg = b'\xfe!\x13\xff\xbeL\x00\x00\x14C\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x01\x01\x00Q-'

# unknown msgs ------------------------>
# from_msg = b'\xfe\x0b\x16\xff\xbeE\x00\x00\x00\x00\xf4\x01\x00\x00\x00\x00\x01\xb2\x82'
# from_msg = b'\xfe\x0b \xff\xbeE\x00\x00\x00\x00\xf4\x01\x00\x00\x00\x00\x01hT'
from_msg = b'\xfe\x0b\xe5\xff\xbeE\x00\x00\x00\x00\xf4\x01\x00\x00\x00\x00\x01\r@'



for byte in from_msg:
    msg = dec.parse_char(bytes([byte]))  # Преобразуем каждый элемент в байт
    if msg:
        # Проверяем тип сообщения
        print(msg)
        if msg.get_type() == "MANUAL_CONTROL":
            # Извлекаем значения полей
            x = msg.x
            y = msg.y
            z = msg.z
            print(f"x: {x}, y: {y}, z: {z}")


MANUAL_CONTROL {target : 1, x : 0, y : 0, z : 500, r : 0, buttons : 0}
x: 0, y: 0, z: 500


In [ ]:
import struct
temp = bytearray(b'\xfe\x0b\xe5\xff\xbeE')
struct.unpack("<cBBBBB", temp)
# temp.decode('utf-8')

In [5]:
from pymavlink import mavutil

# Создаем объект MAVLink
master = mavutil.mavlink.MAVLink(None)  # None, потому что мы не используем поток данных

# Сообщение в байтах
from_msg = b'\xfe\x0b\xe5\xff\xbeE\x00\x00\x00\x00\xf4\x01\x00\x00\x00\x00\x01\r@'

# Парсинг сообщения
for byte in from_msg:
    msg = master.parse_char(bytes([byte]))  # Преобразуем каждый элемент в байт
    if msg:
        # Проверяем тип сообщения
        if msg.get_type() == "MANUAL_CONTROL":
            # Извлекаем значения полей
            x = msg.x
            y = msg.y
            z = msg.z
            print(f"x: {msg.x}, y: {msg.y}, z: {msg.z}, r: {msg.r}, buttons : {msg.buttons}")

x: 0, y: 0, z: 500, r: 0, buttons : 0


# Sending mpu datas to qground 

In [2]:
from msgs import MAVLink_scaled_imu_message

class accel:
    x = 0.1
    y = 1.1
    z = 2.1

class gyro:
    x = 6.1
    y = 2.1
    z = 0.1

class mag:
    x = 8.1
    y = 6.1
    z = 1.1

class mpu:
    accel = accel
    gyro = gyro
    mag = mag


agm = MAVLink_scaled_imu_message(0, int(mpu.accel.x), int(mpu.accel.y), int(mpu.accel.z), int(mpu.gyro.x), int(mpu.gyro.y), int(mpu.gyro.z), int(mpu.mag.x), int(mpu.mag.y), int(mpu.mag.z))
print('messege = {}'.format(agm.pack()))


messege = b'\xfe\x15\x00\x01\x01\x1a\x00\x00\x00\x00\x00\x00\x01\x00\x02\x00\x06\x00\x02\x00\x00\x00\x08\x00\x06\x00\x01\x17v'
